In [1]:
import pandas as pd
import joblib

log_model = joblib.load("../models/logistic_regression.pkl")
nb_model = joblib.load("../models/naive_bayes.pkl")
vectorizer = joblib.load("../models/tfidf_vectorizer.pkl")

print("Models and vectorizer loaded sucessfully.")

Models and vectorizer loaded sucessfully.


In [2]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/processed/reddit_clean.csv")

X = df["clean_text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Test case size:", X_test.shape)

Test case size: (465418,)


In [3]:
X_test_tfidf = vectorizer.transform(X_test)

log_preds = log_model.predict(X_test_tfidf)
nb_preds = nb_model.predict(X_test_tfidf)

print("Predictions generated.")

Predictions generated.


In [4]:
error_df = pd.DataFrame({
    "text": X_test.values,
    "true_label": y_test.values,
    "log_pred": log_preds,
    "nb_pred": nb_preds
})

error_df.head()

,text,true_label,log_pred,nb_pred
0,grandmother help mom every often spend time tr...,0.0,0,0
1,many people believe dumb star judge personalit...,0.0,0,0
2,bored need social interaction,0.0,0,0
3,reddit premium gay,0.0,0,0
4,little trolling,0.0,0,0


In [5]:
# Analyzing False Negative
log_fn = error_df[
    (error_df["true_label"] == 1) &
    (error_df["log_pred"] == 0)
]

print("Logistic Regression False Negatives:", len(log_fn))
log_fn.sample(5)

Logistic Regression False Negatives: 9236


,text,true_label,log_pred,nb_pred
424546,school make suicidal school country opened bac...,1.0,0,0
192133,surrender month sweep,1.0,0,0
202512,reflecting something happened six school stand...,1.0,0,0
116853,cry song build home cinematic orchestra safe s...,1.0,0,0
243648,know wether,1.0,0,0


In [6]:
nb_fn = error_df[
    (error_df["true_label"] == 1) &
    (error_df["nb_pred"] == 0)
]

print("Naive Bayes False Negatives:", len(nb_fn))
nb_fn.sample(5)

Naive Bayes False Negatives: 22782


,text,true_label,log_pred,nb_pred
141005,ive eaten today apple fucking long ive hated b...,1.0,1,0
172588,really wonder work trust soon leave like shit ...,1.0,1,0
294327,comic found want look,1.0,0,0
34251,really making day barely mustered strength get...,1.0,1,0
383206,hey go back previous comment get idea story ba...,1.0,1,0


### False Negative Analysis

False negatives represent depressed posts incorrectly classified as normal.

- Logistic Regression shows **fewer false negatives**, meaning it detects
  depression more reliably.
- Naive Bayes misses more depressed posts, making it riskier for
  mental-health-related applications.

In depression detection, minimizing false negatives is more important
than minimizing false positives.

### Final Model Choice

Although both models achieve similar accuracy (~92%):

- **Logistic Regression** achieves much higher recall for depressed posts.
- **Naive Bayes** is more conservative but misses many depressed users.

For depression detection, **Logistic Regression is the preferred model**
because it minimizes false negatives, which is critical in healthcare
and mental well-being applications.

### *Model Explainability*

In [7]:
# This gives us the actual words used by TF-IDF.
feature_names = vectorizer.get_feature_names_out()

print("Number of features:", len(feature_names))
feature_names[:20]

Number of features: 15000


array(['aa', 'ab', 'abandon', 'abandoned', 'abandonment', 'ability',
       'able', 'able afford', 'able anything', 'able feel', 'able find',
       'able get', 'able go', 'able handle', 'able help', 'able keep',
       'able live', 'able make', 'able move', 'able pay'], dtype=object)

In [8]:
coefficients = log_model.coef_[0]

print("Coefficient shape:", coefficients.shape)

# One coefficient per word
# Positive → depressed
# Negative → normal

Coefficient shape: (15000,)


In [9]:
coef_df = pd.DataFrame({
    "word": feature_names,
    "weight": coefficients
})

coef_df.head()

,word,weight
0,aa,-0.530853
1,ab,-1.503518
2,abandon,1.918048
3,abandoned,1.706644
4,abandonment,0.841699


In [10]:
top_depressed_words = coef_df.sort_values(
    by="weight", ascending=False
).head(20)

top_depressed_words

,word,weight
2669,depression,13.664051
12400,suicide,9.621884
12393,suicidal,8.581851
2650,depressed,8.088554
12500,sw,7.570842
484,antidepressant,6.530498
12413,suicidewatch,6.391375
8210,mg,6.356605
10114,psychiatrist,6.270974
9689,pill,6.097288


In [11]:
top_normal_words = coef_df.sort_values(
    by="weight", ascending=True
).head(20)

top_normal_words

,word,weight
12700,teenager,-13.310911
2293,crush,-10.187798
4190,flair,-9.405389
8159,meme,-8.483479
8271,minecraft,-8.427723
2900,dm,-8.058720
2854,discord,-7.837713
13366,tldr,-7.733188
6509,karma,-6.956771
12695,teen,-6.709778


### Model Explainability (Logistic Regression)

The Logistic Regression model assigns a weight to each word based on its
contribution to predicting depression.

- Words with **positive weights** increase the likelihood of a post being
  classified as depressed.
- Words with **negative weights** reduce the likelihood of depression.

#### Depression-Indicative Words
The model strongly associates words such as *worthless*, *hopeless*,
*alone*, and *tired* with depressed posts.

#### Normal / Non-Depressed Words
Words like *happy*, *excited*, *love*, and *great* are associated with
normal mental states and reduce the probability of depression.

This makes the model interpretable and suitable for sensitive applications
like mental health analysis.